In [ ]:
#MODELO PREDITIVO INDIVIDUAL RELATIVO À SIMULATED ANNEALING - MODELO DE REGRESSÃO DE AFINAÇÃO DOS PARÂMETROS --------------------------------------

import pandas as pd
import os
import random
import math
import numpy as np 
from matplotlib import pyplot as plt
import xlwings as xw
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression

#EXCEL - CAMINHO DO FICHEIRO 
file_path = r"D:\Documentos\...\DataSet_Regressão_SA.xlsx"
dados_problema= pd.read_excel(file_path)
print (dados_problema)


     MH  NJOBS  NMACHINES  CMAXOPT OBJFUNC  CMAX  TIMEEXEC  INITTEMP  \
0    SA      6          6       55    Cmax    59      0.09        15   
1    SA     10          5      590      WT   737      0.13        15   
2    SA     10          5      666      WT   740      0.15        15   
3    SA     10         10     1005      WT  1362      0.30        15   
4    SA     10          5      593      WT   593      0.15        15   
..   ..    ...        ...      ...     ...   ...       ...       ...   
162  SA     50         10     2972    Cmax  5191      7.55        15   
163  SA     50         10     3104    Cmax  5495      5.27        15   
164  SA     10         10      848      WT  1037      0.22        14   
165  SA     10         10      943      WT  1078      0.27        14   
166  SA     10         10      944      WT  1411      0.24        14   

     NUMITERAK  SA_STOPCRIT  
0           10           25  
1           15           38  
2           15           38  
3           15 

In [2]:
#DIVIDIR DADOS EM TREINO E TESTE (RETIRAR COLONA CMAX, MH E TIMEEXEC (NÃO É UMA FEATURE))
dados_problema=dados_problema.drop(columns=['CMAX'])
dados_problema=dados_problema.drop(columns=['MH'])
dados_problema=dados_problema.drop(columns=['TIMEEXEC'])
#print (dados_problema)
X = dados_problema.drop(columns=['NUMITERAK', 'SA_STOPCRIT', 'INITTEMP'])  #VARIÁVEIS INDEPENDENTES
y = dados_problema[['NUMITERAK', 'SA_STOPCRIT', 'INITTEMP']] #VARIÁVEL DEPENDENTE

#COLUNA OBJFUNX: PASSAR CMAX = 0 E WT=1 - NECESSÁRIO EM ALGUMAS TÉCNICAS
X['OBJFUNC'] = X['OBJFUNC'].map({'Cmax': 0, 'WT': 1})

#DIVIDIR OS DADOS
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [3]:
#ÁRVORE DE DECISÃO ------------------------------------------------------------------------------------------------------------------------------

#FUNÇÃO PARA O CÁLCULO DAS MÉTRICAS DE AVALIAÇÃO
def evaluate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
    mape = np.mean(np.abs((y_true - y_pred) / y_true), axis=0) * 100
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')
    return mse, rmse, mae, mape, r2

#CRIAR MODELO DE ÁRVORE DE DECISÃO
base_model = DecisionTreeRegressor(random_state=42)
model = MultiOutputRegressor(base_model)

#DEFINIR PARÂMETROS
param_grid = {
    'estimator__criterion': ['squared_error', 'absolute_error'],
    'estimator__max_depth': [3, 5, 10, 15, 20, None],
    'estimator__min_samples_split': [2, 5, 10, 15],
    'estimator__min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 10, 15]
}

#GRIDSEARCH COM VALIDAÇÃO CRUZADA 
grid_search = GridSearchCV(
    model,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

#TREINAR O MODELO À PROCURA DOS MELHOR HIPERPARÂMETROS
grid_search.fit(X_train, y_train)

#OBTER O MELHOR MODELO
best_model = grid_search.best_estimator_
print("\nMelhores Hiperparâmetros:", grid_search.best_params_)

#PREVISÃO E CALCULO DAS MÉTRICAS DE AVALIAÇÃO
y_test_pred = best_model.predict(X_test)
mse_test, rmse_test, mae_test, mape_test, r2_test = evaluate_metrics(y_test, y_test_pred)

'''
#RESULTADOS DAS MÉTRICAS DE AVALIAÇÃO DE CADA VARIÁVEL DEPENDENTE
print("=== Resultados no CONJUNTO DE TESTE ===")
for i, target in enumerate(['NUMITERAK', 'SA_STOPCRIT','INITTEMP']):
    print(f"\nVariável: {target}")
    print(f"MSE: {mse_test[i]:.4f}")
    print(f"RMSE: {rmse_test[i]:.4f}")
    print(f"MAE: {mae_test[i]:.4f}")
    print(f"MAPE: {mape_test[i]:.2f}%")
    print(f"R²: {r2_test[i]:.4f}")
'''

#RESULTADOS GERAIS - MÉDIA DOS RESULTADOS DAS DUAS VARIÁVEIS (À EXCEÇÃO DO RMSE) 
mse_test_mean = np.mean(mse_test)
rmse_test_mean = np.sqrt(mse_test_mean)
mae_test_mean = np.mean(mae_test)
mape_test_mean = np.mean(mape_test)
r2_test_mean = np.mean(r2_test)

print("\n=== RESULTADO GERAL (MÉDIA TESTE) ===")
print(f"MSE: {mse_test_mean:.4f}")
print(f"RMSE: {rmse_test_mean:.4f}")
print(f"MAE: {mae_test_mean:.4f}")
print(f"MAPE: {mape_test_mean:.2f}%")
print(f"R²: {r2_test_mean:.4f}")

#IMPORTANCIA DAS FEATURES
#CALCULAR A MÉDIA DA IMPORTÂNCIA DE CADA VARIAVEL DEPENDENTE 
importances = np.mean([est.feature_importances_ for est in best_model.estimators_], axis=0)
feature_names = X.columns
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})
importance_df['Importance'] = importance_df['Importance'].round(4)
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print("\n=== Importância das Features (Média entre os modelos) ===")
print(importance_df)


Melhores Hiperparâmetros: {'estimator__criterion': 'squared_error', 'estimator__max_depth': 5, 'estimator__min_samples_leaf': 5, 'estimator__min_samples_split': 15}

=== RESULTADO GERAL (MÉDIA TESTE) ===
MSE: 24.2258
RMSE: 4.9220
MAE: 1.3767
MAPE: 3.04%
R²: 0.5037

=== Importância das Features (Média entre os modelos) ===
     Feature  Importance
0      NJOBS      0.8492
2    CMAXOPT      0.1496
1  NMACHINES      0.0012
3    OBJFUNC      0.0000


In [4]:
#RANDOM FOREST ------------------------------------------------------------------------------------------------------------------------------------------

#FUNÇÃO PARA CALCULAR AS MÉTRICAS DE AVALIAÇÃO
def evaluate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
    mape = np.mean(np.abs((y_true - y_pred) / y_true), axis=0) * 100
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')
    return mse, rmse, mae, mape, r2

#CRIAR MODELO RANDOM FOREST
base_model = RandomForestRegressor(random_state=42)
model = MultiOutputRegressor(base_model)

#DEFINIR PARÂMETROS
param_grid = {
    'estimator__n_estimators': [50, 100, 200, 300, 400],
    'estimator__max_depth': [None, 3, 5, 7, 10, 20, 30],
    'estimator__min_samples_split': [2, 5, 10, 15],
    'estimator__min_samples_leaf': [1, 2, 4, 6],
    'estimator__max_features': [None, 'sqrt', 'log2'],
}

#GRIDSEARCH COM VALIDAÇÃO CRUZADA
grid_search = GridSearchCV(
    model,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

#TREINAR O MODELO À PROCURA DOS MELHOR HIPERPARÂMETROS
grid_search.fit(X_train, y_train)

#OBTER O MELHOR MODELO
best_model = grid_search.best_estimator_ 
print("\nMelhores Hiperparâmetros:", grid_search.best_params_)

#PREVISÃO E CALCULO DAS MÉTRICAS DE AVALIAÇÃO
y_test_pred = best_model.predict(X_test)
mse_test, rmse_test, mae_test, mape_test, r2_test = evaluate_metrics(y_test, y_test_pred)

'''
#RESULTADOS DAS MÉTRICAS DE AVALIAÇÃO DE CADA VARIÁVEL DEPENDENTE
print("=== Resultados no CONJUNTO DE TESTE (Random Forest) ===")
for i, target in enumerate(['NUMPART', 'NUMITERA']):
    print(f"\nVariável: {target}")
    print(f"MSE: {mse_test[i]:.4f}")
    print(f"RMSE: {rmse_test[i]:.4f}")
    print(f"MAE: {mae_test[i]:.4f}")
    print(f"MAPE: {mape_test[i]:.2f}%")
    print(f"R²: {r2_test[i]:.4f}")
'''

#RESULTADOS GERAIS - MÉDIA DOS RESULTADOS DAS DUAS VARIÁVEIS (À EXCEÇÃO DO RMSE) 
mse_test_mean = np.mean(mse_test)
rmse_test_mean = np.sqrt(mse_test_mean)
mae_test_mean = np.mean(mae_test)
mape_test_mean = np.mean(mape_test)
r2_test_mean = np.mean(r2_test)

print("\n=== RESULTADO GERAL (MÉDIA TESTE) ===")
print(f"MSE: {mse_test_mean:.4f}")
print(f"RMSE: {rmse_test_mean:.4f}")
print(f"MAE: {mae_test_mean:.4f}")
print(f"MAPE: {mape_test_mean:.2f}%")
print(f"R²: {r2_test_mean:.4f}")


#IMPORTANCIA DAS FEATURES
#CALCULAR A MÉDIA DA IMPORTÂNCIA DE CADA VARIAVEL DEPENDENTE 
importances = np.mean([est.feature_importances_ for est in best_model.estimators_], axis=0)
feature_names = X.columns

importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})
importance_df['Importance'] = importance_df['Importance'].round(4)
importance_df = importance_df.sort_values(by='Importance', ascending=False)
print("\n=== Importância das Features (Random Forest - Média entre os modelos) ===")
print(importance_df)



Melhores Hiperparâmetros: {'estimator__max_depth': None, 'estimator__max_features': None, 'estimator__min_samples_leaf': 2, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 50}

=== RESULTADO GERAL (MÉDIA TESTE) ===
MSE: 33.5273
RMSE: 5.7903
MAE: 1.5596
MAPE: 3.16%
R²: 0.4914

=== Importância das Features (Random Forest - Média entre os modelos) ===
     Feature  Importance
2    CMAXOPT      0.5200
0      NJOBS      0.4386
1  NMACHINES      0.0253
3    OBJFUNC      0.0161


In [5]:
#REGRESSÃO LINEAR SIMPLES ----------------------------------------------------------------------------------------------------------

#FUNÇÃO PARA CALCULAR AS MÉTRICAS DE AVALIAÇÃO
def evaluate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
    mape = np.mean(np.abs((y_true - y_pred) / y_true), axis=0) * 100
    r2 = r2_score(y_true, y_pred, multioutput='raw_values')
    return mse, rmse, mae, mape, r2

#CRIAR MODELO DE REGRESSÃO LINEAR
base_model = LinearRegression()
model = MultiOutputRegressor(base_model)

#TREINAR MODELO
model.fit(X_train, y_train)

#PREVISÕES
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

#AVALIAR CONJUNTO DE TESTE
mse_test, rmse_test, mae_test, mape_test, r2_test = evaluate_metrics(y_test, y_test_pred)

'''
#RESULTADOS  DAS MÉTRICAS DE AVALIAÇÃO DE CADA VARIÁVEL DEPENDENTE
print("=== Resultados no CONJUNTO DE TESTE (Regressão Linear Simples) ===")
for i, target in enumerate(['NUMITERAK', 'SA_STOPCRIT', 'INITTEMP']):
    print(f"\nVariável: {target}")
    print(f"MSE: {mse_test[i]:.4f}")
    print(f"RMSE: {rmse_test[i]:.4f}")
    print(f"MAE: {mae_test[i]:.4f}")
    print(f"MAPE: {mape_test[i]:.2f}%")
    print(f"R²: {r2_test[i]:.4f}")
'''

#RESULTADOS GERAIS - MÉDIA DOS RESULTADOS DAS DUAS VARIÁVEIS (À EXCEÇÃO DO RMSE) 
mse_test_mean = np.mean(mse_test)
rmse_test_mean = np.sqrt(mse_test_mean)
mae_test_mean = np.mean(mae_test)
mape_test_mean = np.mean(mape_test)
r2_test_mean = np.mean(r2_test)

print("\n=== RESULTADO GERAL (MÉDIA TESTE) ===")
print(f"MSE: {mse_test_mean:.4f}")
print(f"RMSE: {rmse_test_mean:.4f}")
print(f"MAE: {mae_test_mean:.4f}")
print(f"MAPE: {mape_test_mean:.2f}%")
print(f"R²: {r2_test_mean:.4f}")



=== RESULTADO GERAL (MÉDIA TESTE) ===
MSE: 39.6032
RMSE: 6.2931
MAE: 3.4210
MAPE: 8.80%
R²: 0.4589
